<a href="https://colab.research.google.com/github/m-manuelmussa/Desenvolvimento-de-modelos-de-machine-learning-para-predizer-novos-inibidores-da-TK-HER2/blob/main/1_Preparo_do_banco_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Desenvolvimento de modelos de machine learning baseados em QSAR-2D para a predição de novos candidatos a fármacos TKI-HER2 para o tratamento de câncer da mama***

---

*@Micliete_Mussa*

# ***1. Preparo do banco de dados***

### **Pipeline**
1. Configuração do ambiente de trabalho
2. Selecção do dataset de compostos inibidores da TK-HER2
3. Seleção dos compostos com dados de IC₅₀
4. Limpeza dos dados (sem duplicatas, IC₅₀ e SMILES ausentes)
5. Dataset final limpo e pronto para análise

### ***1. Configuração do ambiente de trabalho***

In [1]:
#Instalação de frameworks necessários
!pip install chembl_webresource_client #API Cliente do CHEMBL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 919.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.5 MB/s eta 0:00:00


In [2]:
# Importação de frameworks necessários
from chembl_webresource_client.new_client import new_client #Criação da API Cliente do CHEMBL
import pandas as pd # Para tabulação de dados
from tqdm import tqdm  # barra de progresso

### ***2. Selecção do dataset de compostos inibidores da TK-HER2***

In [4]:
# Seleção do alvo HER2 (tirosina-quinase) com base no CHEMBL ID
alvo_especifico = "CHEMBL1824"  # HER2/ERBB2

# Buscar atividades biológicas relacionadas ao alvo e os compostos
actividades = new_client.activity.filter(target_chembl_id=alvo_especifico).only(
    ["molecule_chembl_id", "standard_type", "standard_relation",
     "standard_value", "standard_units"]
)

# Criar a lista do QuerySet (permite manipular os dados requisitados)
actividades_list = list(actividades)

# Criar DataFrame com compostos
df = pd.DataFrame(actividades_list)

In [6]:
display(df)

,molecule_chembl_id,relation,standard_relation,standard_type,standard_units,standard_value,type,units,value
0,CHEMBL68920,=,=,IC50,nM,300.0,IC50,uM,0.3
1,CHEMBL68920,=,=,IC50,nM,2500.0,IC50,uM,2.5
2,CHEMBL69960,=,=,IC50,nM,400.0,IC50,uM,0.4
3,CHEMBL69960,=,=,IC50,nM,1210.0,IC50,uM,1.21
4,CHEMBL67057,=,=,IC50,nM,100.0,IC50,uM,0.1
...,...,...,...,...,...,...,...,...,...
7958,CHEMBL554,=,=,Kd,nM,7.0,Kd,nM,7.0
7959,CHEMBL3651966,=,=,% of control,%,92.0,% of control,%,92.0
7960,CHEMBL3393348,=,=,% of control,%,73.0,% of control,%,73.0
7961,CHEMBL4877117,=,=,% of control,%,-0.819,% of control,%,-0.819


In [7]:
# Buscar informações do alvo (nome + organismo)
alvo_info = new_client.target.filter(target_chembl_id=alvo_especifico).only(
    ["pref_name", "organism"]
)
alvo_info = list(alvo_info)[0]  # pega o primeiro registro (único para este ID)

# Adicionar colunas "Alvo" e "Organismo" no DataFrame
df["Alvo"] = alvo_info["pref_name"]
df["Organismo"] = alvo_info["organism"]

# Remover duplicatas (compostos podem aparecer várias vezes em bioensaios)
df_final = df.drop_duplicates(subset=["molecule_chembl_id"]).reset_index(drop=True)

# Contar compostos únicos (percorre cada linha para contabilizar o somatório)
total_compostos_unicos = df_final.shape[0]

# Exibir resultado
display(df_final.head(10))  # mostra 10 primeiras linhas
print(f"DATASET INICIAL: {total_compostos_unicos} COMPOSTOS testados contra {alvo_info['pref_name']} ({alvo_info['organism']})")

,molecule_chembl_id,relation,standard_relation,standard_type,standard_units,standard_value,type,units,value,Alvo,Organismo
0,CHEMBL68920,=,=,IC50,nM,300.0,IC50,uM,0.3,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
1,CHEMBL69960,=,=,IC50,nM,400.0,IC50,uM,0.4,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
2,CHEMBL67057,=,=,IC50,nM,100.0,IC50,uM,0.1,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
3,CHEMBL65848,>,>,IC50,nM,5000.0,IC50,uM,5.0,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
4,CHEMBL69629,=,=,IC50,nM,100.0,IC50,uM,0.1,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
5,CHEMBL66570,=,=,IC50,nM,100.0,IC50,uM,0.1,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
6,CHEMBL305194,>,>,IC50,nM,5000.0,IC50,uM,5.0,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
7,CHEMBL67003,=,=,IC50,nM,100.0,IC50,uM,0.1,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
8,CHEMBL305246,=,=,IC50,nM,200.0,IC50,uM,0.2,Receptor protein-tyrosine kinase erbB-2,Homo sapiens
9,CHEMBL69966,=,=,IC50,nM,40.0,IC50,uM,0.04,Receptor protein-tyrosine kinase erbB-2,Homo sapiens


DATASET INICIAL: 5577 COMPOSTOS testados contra Receptor protein-tyrosine kinase erbB-2 (Homo sapiens)


### ***3. Seleção dos compostos com dados de IC₅₀***

In [10]:
# Buscar atividades com IC50 contra HER2
atividades_ic50 = new_client.activity.filter(
    target_chembl_id=alvo_especifico,
    standard_type="IC50"
).only(["molecule_chembl_id", "standard_value"])

# Contar compostos únicos com IC50
compostos_unicos = set()
for atividade in atividades_ic50:
    if atividade.get("standard_value") is not None:
        compostos_unicos.add(atividade["molecule_chembl_id"])

In [11]:
# Criar lista de dicionários com valores válidos de IC50
atividades_ic50_list = [
    {"molecule_chembl_id": a["molecule_chembl_id"], "IC50": a["standard_value"]}
    for a in atividades_ic50
    if a.get("standard_value") is not None
]

# Criar DataFrame
df_ic50 = pd.DataFrame(atividades_ic50_list)

# Exibir as primeiras linhas
display(df_ic50.head(10))

# Contagem de compostos únicos
print(f"DATASET SELECIONADO: {df_ic50['molecule_chembl_id'].nunique()} COMPOSTOS")

,molecule_chembl_id,IC50
0,CHEMBL68920,300.0
1,CHEMBL68920,2500.0
2,CHEMBL69960,400.0
3,CHEMBL69960,1210.0
4,CHEMBL67057,100.0
5,CHEMBL65848,5000.0
6,CHEMBL65848,2780.0
7,CHEMBL69629,100.0
8,CHEMBL69629,1240.0
9,CHEMBL66570,100.0


DATASET SELECIONADO: 2729 COMPOSTOS


### ***4. Limpeza dos dados (sem duplicatas, IC₅₀ e SMILES ausentes)***

In [13]:
# Transformar em lista e DataFrame
atividades_list = list(atividades_ic50)
df = pd.DataFrame(atividades_list)

# Remover dados sem IC50
df = df.dropna(subset=["standard_value"])
df["IC50_nM"] = df["standard_value"].astype(float)

# Buscar os SMILES das moléculas
tqdm.pandas()

molecule = new_client.molecule
def get_smiles(chembl_id):
    try:
        mol = molecule.get(chembl_id)
        if mol.get("molecule_structures"):
            return mol["molecule_structures"]["canonical_smiles"]
    except:
        return None

# Adicionar coluna de SMILES
df["canonical_smiles"] = df["molecule_chembl_id"].progress_apply(get_smiles)

# Remover compostos sem SMILES ou IC50
df_clean = df.dropna(subset=["canonical_smiles", "IC50_nM"])

# Remover duplicatas por SMILES
df_clean = df_clean.drop_duplicates(subset="canonical_smiles")

100%|██████████| 3228/3228 [00:08<00:00, 377.37it/s]


### ***5. Dataset final limpo e pronto para análise***

In [14]:
# 1. Resultado final
df_clean.head()
display(df_clean.head(10))
print(f"\nDATASET FINAL: {len(df_clean)} COMPOSTOS")

,molecule_chembl_id,standard_value,value,IC50_nM,canonical_smiles
0,CHEMBL68920,300.0,0.3,300.0,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...
2,CHEMBL69960,400.0,0.4,400.0,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...
4,CHEMBL67057,100.0,0.1,100.0,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...
6,CHEMBL65848,5000.0,5.0,5000.0,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...
8,CHEMBL69629,100.0,0.1,100.0,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncn...
10,CHEMBL66570,100.0,0.1,100.0,CCN1CCN(C(=O)c2cc(C)c(/C=C3\C(=O)Nc4ncnc(Nc5cc...
12,CHEMBL305194,5000.0,5.0,5000.0,Cc1cc(C(=O)O)[nH]c1/C=C1\C(=O)Nc2ncnc(Nc3ccc(F...
14,CHEMBL67003,100.0,0.1,100.0,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...
16,CHEMBL305246,200.0,0.2,200.0,Cc1cc(C(=O)N2CCN(C)CC2)[nH]c1/C=C1\C(=O)Nc2ncn...
18,CHEMBL69966,40.0,0.04,40.0,C#Cc1cccc(Nc2ncnc3c2/C(=C/c2[nH]c(C(=O)NCCN4CC...



DATASET FINAL: 2725 COMPOSTOS


In [15]:
# 2. Selecionar colunas desejadas e renomear
df_export = df_clean[["molecule_chembl_id", "canonical_smiles", "IC50_nM"]].rename(columns={
    "molecule_chembl_id": "ChEMBL_ID",
    "canonical_smiles": "SMILES",
    "IC50_nM": "IC50_nM"
})

# 3. Exportar para CSV com separador ";"
df_export.to_csv("Dataset_final.csv", sep=";", index=False)

print("Arquivo CSV salvo como 'Dataset_final.csv'")

Arquivo CSV salvo como 'Dataset_final.csv'


# ***FIM***